In [54]:
import pandas as pd
import numpy as np
from scipy.optimize import brentq
import math
import scipy.stats as si
import matplotlib.pyplot as plt
import scipy.stats as stats
import re

file_names = [
    "TSM 2013-2014.xlsx",
    "TSM 2014-2015.xlsx",
    "TSM 2015-2016.xlsx",
    "TSM 2016-2017.xlsx",
    "TSM 2017-2018 (1).xlsx",
    "TSM 2018-2019.xlsx",
    "TSM 2019-2020.xlsx",
    "TSM 2020-2021.xlsx",
    "TSM 2021-2022.xlsx",
    "TSM 2022-2023.xlsx",
    "TSM 2023-2024.xlsx",
    "TSM 2024-2025.xlsx"
]


df_combined = pd.concat([pd.read_excel(file) for file in file_names], ignore_index=True)
df_combined

,Time Interval,Close,Net Chg,Open,High,Low,Tick Count,Volume
0,Summary,17.830,0.040,18.0000,20.3000,15.700,8447324.0,2489180660
1,27FEB2013_00:00:00.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09:30 - 10:30,18.130,0.340,18.0000,18.1650,17.990,13797.0,4874289
3,10:30 - 11:30,18.200,0.070,18.1300,18.2025,18.110,5596.0,1537670
4,11:30 - 12:30,18.265,0.065,18.2000,18.2800,18.180,3722.0,986223
...,...,...,...,...,...,...,...,...
24103,11:30 - 12:30,177.265,-0.845,178.0800,178.1300,176.520,13875.0,2229912
24104,12:30 - 13:30,173.700,-3.565,177.2625,177.3799,173.650,14151.0,2346877
24105,13:30 - 14:30,173.610,-0.090,173.6900,174.8200,173.310,18308.0,2828659
24106,14:30 - 15:30,171.495,-2.115,173.6200,176.8100,171.070,30381.0,5040781


In [63]:
def format_time_intervals(df):
    current_date = None  
    rows_to_keep = []  

    for _, row in df.iterrows():
        if row.count() == 1:  
            current_date = row.iloc[0]  
        else:
            if current_date:
                row["Time Interval"] = f"{current_date[0:9]} {row['Time Interval']}"
            rows_to_keep.append(row)
    formatted_df = pd.DataFrame(rows_to_keep)

    return formatted_df


print(format_time_intervals(df_combined))
df = (format_time_intervals(df_combined))

                 Time Interval    Close  Net Chg      Open      High      Low  \
0                      Summary   17.830    0.040   18.0000   20.3000   15.700   
2      27FEB2013 09:30 - 10:30   18.130    0.340   18.0000   18.1650   17.990   
3      27FEB2013 10:30 - 11:30   18.200    0.070   18.1300   18.2025   18.110   
4      27FEB2013 11:30 - 12:30   18.265    0.065   18.2000   18.2800   18.180   
5      27FEB2013 12:30 - 13:30   18.280    0.015   18.2700   18.3200   18.240   
...                        ...      ...      ...       ...       ...      ...   
24103  03MAR2025 11:30 - 12:30  177.265   -0.845  178.0800  178.1300  176.520   
24104  03MAR2025 12:30 - 13:30  173.700   -3.565  177.2625  177.3799  173.650   
24105  03MAR2025 13:30 - 14:30  173.610   -0.090  173.6900  174.8200  173.310   
24106  03MAR2025 14:30 - 15:30  171.495   -2.115  173.6200  176.8100  171.070   
24107  03MAR2025 15:30 - 16:30  173.100    1.605  171.5050  173.4500  171.505   

       Tick Count      Volu

In [73]:
df["Net Chg"] = pd.to_numeric(df["Net Chg"], errors="coerce")
df = df.dropna(subset=["Net Chg"])

mean_change = df["Net Chg"].mean()
std_change = df["Net Chg"].std()

negative_threshold = mean_change - (2 * std_change)

filtered_df = df[df["Net Chg"] < negative_threshold]
filtered_df

,Time Interval,Close,Net Chg,Open,High,Low,Tick Count,Volume
10063,28FEB2018 Summary,39.050,-4.305,43.3500,45.7500,34.2145,9238155.0,1740677694
10337,19APR2018 09:30 - 10:30,39.520,-2.410,39.9300,40.0800,39.3600,35056.0,8037142
14105,09MAR2020 09:30 - 10:30,52.790,-2.270,51.5500,54.1400,51.0600,13296.0,2238313
14129,12MAR2020 09:30 - 10:30,49.900,-2.350,49.6600,50.2370,48.7400,13660.0,2498316
14145,16MAR2020 09:30 - 10:30,46.720,-5.510,46.3400,47.0000,45.2000,11395.0,2191117
...,...,...,...,...,...,...,...,...
24089,27FEB2025 13:30 - 14:30,184.960,-2.940,187.8600,187.9600,184.8600,7245.0,1281768
24090,27FEB2025 14:30 - 15:30,182.715,-2.245,184.9400,185.0000,182.3200,17055.0,2517958
24094,28FEB2025 10:30 - 11:30,179.660,-2.110,181.7350,182.1600,179.5600,14772.0,2324474
24104,03MAR2025 12:30 - 13:30,173.700,-3.565,177.2625,177.3799,173.6500,14151.0,2346877


In [76]:
def getmonthyear(df):
    allmonthyear = set()
    for _,row in df.iterrows():
        monthyear = row["Time Interval"][2:9]
        if monthyear not in allmonthyear:
            allmonthyear.add(monthyear)

    return list(allmonthyear)

l = getmonthyear(filtered_df)
print(l)

['OCT2020', 'MAR2023', 'OCT2021', 'FEB2024', 'FEB2018', 'JUL2022', 'MAY2024', 'JAN2021', 'SEP2022', 'NOV2024', 'MAR2024', 'JUL2024', 'NOV2022', 'DEC2022', 'FEB2023', 'MAR2020', 'MAR2021', 'JUN2021', 'AUG2023', 'AUG2024', 'SEP2023', 'JUL2023', 'JAN2024', 'FEB2022', 'JAN2025', 'NOV2020', 'AUG2022', 'SEP2020', 'NOV2021', 'JAN2023', 'AUG2021', 'DEC2024', 'APR2022', 'JUL2021', 'FEB2021', 'APR2023', 'SEP2024', 'MAR2022', 'FEB2025', 'MAY2023', 'MAR2025', 'APR2024', 'AUG2020', 'JUN2022', 'OCT2024', 'OCT2023', 'DEC2023', 'JAN2022', 'JUL2020', 'SEP2021', 'JUN2024', 'MAY2021', 'MAY2022', 'APR2018', 'APR2021', 'OCT2022']
